In [2]:
%logstop
%logstart -ortq ~/.logs/ml.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
from static_grader import grader

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [3]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
File ‘./ml-data/providers-train.csv’ already there; not retrieving.

File ‘./ml-data/providers-metadata.csv’ already there; not retrieving.



We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [4]:
import numpy as np
import pandas as pd

In [5]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation


In [6]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

In [7]:
data.columns

Index(['PROVNUM', 'PROVNAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'PHONE',
       'COUNTY_SSA', 'COUNTY_NAME', 'BEDCERT', 'RESTOT', 'INHOSP',
       'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL',
       'ADJ_TOTAL', 'OWNERSHIP', 'CERTIFICATION', 'CYCLE_1_DEFS',
       'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP', 'CYCLE_1_DEFS_SCORE',
       'CYCLE_1_NUMREVIS', 'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE',
       'CYCLE_1_SURVEY_DATE', 'CYCLE_2_SURVEY_DATE', 'FINE_CNT', 'FINE_TOT',
       'CYCLE_2_TOTAL_SCORE'],
      dtype='object')

In [8]:
fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

In [9]:
cycle_2_score

0        16
1        12
2        24
3        24
4        28
         ..
13887    64
13888     0
13889     0
13890     0
13891     0
Name: CYCLE_2_TOTAL_SCORE, Length: 13892, dtype: int64

In [10]:
fine_totals

0        15259
1            0
2            0
3            0
4            0
         ...  
13887        0
13888        0
13889        0
13890        0
13891    70865
Name: FINE_TOT, Length: 13892, dtype: int64

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

In [11]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}

    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        df = pd.concat([X, y], axis = 1).groupby(self.grouper)[y.name].mean()
        self.unknown_avg = y.mean()
        self.group_averages = df.to_dict()        
        return self

    def predict(self, X):
        # Return a list of predicted penalties based on group of samples in X
        Xt = []
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        for x in X[self.grouper]:
            Xt.append(self.group_averages.get(x, self.unknown_avg))
        return np.array(Xt)

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [12]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(memory=None, steps=[('sme', GroupMeanEstimator(grouper='STATE'))],
         verbose=False)

Next we should test that our predict method works.

In [13]:
state_model.predict(data.sample(5))

array([ 2213.51526032,  8054.97761194,  2213.51526032,  7635.16099773,
       43295.23076923])

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [14]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

array([14969.85768788])

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [15]:
grader.score.ml__state_model(state_model.predict)

Your score:  0.9999999999999999


In [16]:
data.sample(5)

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
6618,265326,REPUBLIC NURSING & REHAB,901 EAST HWY 174,REPUBLIC,MO,65738,4177321822,380,Greene,127,...,Medicare and Medicaid,0,0,0,0,0,0,0,2017-09-15,2016-10-14
3199,146096,RIDGEVIEW CARE CENTER,413 RIDGE LANE,OBLONG,IL,62449,6185924228,150,Crawford,55,...,Medicare and Medicaid,1,1,0,0,0,0,0,2017-10-26,2016-09-22
5950,235609,ADVANTAGE LIVING CENTER - ARMADA,22600 ARMADA RIDGE RD,ARMADA,MI,48005,5867845322,490,Macomb,67,...,Medicare and Medicaid,5,5,0,52,1,0,52,2017-07-26,2016-09-08
7344,305077,"LAFAYETTE CENTER, GENESIS HEALTHCARE",93 MAIN STREET,FRANCONIA,NH,3580,6038235502,40,Grafton,72,...,Medicare and Medicaid,3,3,0,16,1,0,16,2017-06-27,2016-08-25
4964,195619,SAVOY CARE CENTER,906 CHERRY STREET,MAMOU,LA,70554,3374680347,190,Evangeline,119,...,Medicare and Medicaid,4,2,2,87,1,0,87,2017-01-27,2016-01-28


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [17]:

# simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

# class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self, columns):
#         self.columns = columns

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         if not isinstance(X, pd.DataFrame):
#             X = pd.DataFrame(X)
#         return X[self.columns]
        
# simple_features = Pipeline([
#     ('cst', ColumnSelectTransformer(simple_cols)),
# ])

In [19]:
from sklearn.impute import SimpleImputer
simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns]
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('fillna', SimpleImputer(missing_values=np.nan, strategy='mean'))
])

**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [20]:
assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data)).any()

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [24]:
from sklearn.ensemble import RandomForestClassifier
simple_features_model = Pipeline([
    ('simple', simple_features),
        # add your estimator here
    ('predictor', RandomForestClassifier(n_estimators=52, max_depth=7.8))

])

In [25]:
simple_features_model.fit(data, fine_counts > 0)

Pipeline(memory=None,
         steps=[('simple',
                 Pipeline(memory=None,
                          steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('fillna',
                                  SimpleImputer(add_indicator=False, copy=True,
                        

In [26]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score:  1.0143458095135365


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [156]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
transformer= OneHotEncoder()
enc = OneHotEncoder()

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('enc', enc)

])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('tsf', transformer)
])

categorical_features = FeatureUnion([
    ('owner',owner_onehot),
    ('cert', cert_onehot)
])

In [157]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data)).any()

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [161]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler

clf = MultinomialNB(alpha=0.01) 

categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    # add your estimator here
    ('scaler', StandardScaler(with_mean=False)),
    ('predictor', clf)
])

In [162]:
categorical_features_model.fit(data, fine_counts > 0)

Pipeline(memory=None,
         steps=[('categorical',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('owner',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('enc',
                                                                  OneHotEncoder(categorical_features=None,
                                                                                categories=None,
                                                                                drop=None,
                                                                                dtype=<class 'numpy.float64'>,
                                                                                handle_unknown='error',
   

In [163]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score:  1.0043781272337378


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [129]:
from sklearn.preprocessing import PolynomialFeatures
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [136]:

business_model = Pipeline([
    ('features', business_features),
    # add your estimator here
    ('polys', PolynomialFeatures()),
    ('regressor', RandomForestClassifier(max_depth=9, min_samples_split=6, n_estimators=60, random_state = 0))
])

In [137]:
business_model.fit(data, fine_counts > 0)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('simple',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                   'RESTOT',
                                                                                                   'INHOSP',
                                                                                                   'CCRC_FACIL',
                                                                                                   'SFF',
                                                                                                   'CHOW_LAST_12MOS',
                                      

In [138]:
grader.score.ml__business_model(positive_probability(business_model))

Your score:  1.0003168133712366


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [164]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return np.array((pd.to_datetime(X[self.t1_col]) - pd.to_datetime(X[self.t2_col])).dt.days).reshape(-1,1)

In [165]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In the cell below we'll collect the cycle 1 survey features.

In [ ]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ...

In [ ]:
from sklearn.ensemble import RandomForestRegressor

survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    # add your estimator here
])

In [ ]:
survey_model.fit(data, cycle_2_score.astype(int))

In [ ]:
grader.score.ml__survey_model(survey_model.predict)

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*